In [1]:
!pwd

D:\Users\drome\gitrepos\spe 06\MannvilleGroup_Strat_Hackathon


In [2]:
# !pip install holoviews
# !pip install lasio

In [3]:
import os
import glob
import pandas as pd
import numpy as np
import holoviews as hv
import collections
import matplotlib.pyplot as plt
%matplotlib inline
import lasio
lasio.__version__

'0.19'

In [4]:
home = "./SPE_006_originalData/OilSandsDB"

In [5]:
# read wells with lat and long coordinates
locations = pd.read_csv("./well_lat_lng.csv", delimiter=',', dtype=str)
locations = locations[['SitID', 'lng', 'lat']]
# df[['color', 'food', 'score']]
locations.head()

,SitID,lng,lat
0,102496,-110.12932,54.785907
1,102497,-110.269446,54.782284
2,102498,-110.186851,54.785892
3,102500,-110.269422,54.829624
4,102501,-110.224832,54.840471


In [6]:
# describe the stratigraphic picks used in Picks
picks_dicts = pd.read_csv(home + "/PICKS_DIC.TXT", delimiter='\t', dtype=str)
picks_dicts.columns = ['HorID', 'Description']
picks_dicts.head()

,HorID,Description
0,1000,mannville
1,2000,t61
2,3000,t51
3,4000,t41
4,5000,t31


In [7]:
# stratigraphic picks and dropping NaN values
picks = pd.read_csv(home + "/PICKS.TXT", delimiter='\t')
picks['SitID'] = picks['SitID'].astype(str)
picks['HorID'] = picks['HorID'].astype(str)
picks['Pick'] = picks['Pick'].str.strip()
picks['Pick'] = pd.to_numeric(picks['Pick'])
picks = picks.dropna()
# picks.columns = ['SitID', 'HorID', 'DEPT', 'Quality']
picks.head()

,SitID,HorID,Pick,Quality
0,102496,1000,321.0,1
4,102496,5000,438.0,2
6,102496,7000,458.0,3
7,102496,9000,468.0,3
8,102496,9500,468.0,3


In [8]:
picks.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20327 entries, 0 to 30701
Data columns (total 4 columns):
SitID      20327 non-null object
HorID      20327 non-null object
Pick       20327 non-null float64
Quality    20327 non-null int64
dtypes: float64(1), int64(1), object(2)
memory usage: 794.0+ KB


In [9]:
# list of wells
wellst = pd.read_csv(home + "/WELLS.TXT", delimiter= '\t', dtype=str)
wellst.head()

,SitID,UWI (AGS),UWI
0,102496,0674010812000,00/12-08-067-01W4/0
1,102497,0674020807000,00/07-08-067-02W4/0
2,102498,0674021109000,00/09-11-067-02W4/0
3,102500,0674022910000,00/10-29-067-02W4/0
4,102501,0674023406000,00/06-34-067-02W4/0


In [10]:
wellst.dtypes

SitID        object
UWI (AGS)    object
UWI          object
dtype: object

In [11]:
# merge wellst, locations, and picks in one dataframe
wl = pd.merge(left=wellst, right=locations, left_on=['SitID'], right_on=['SitID'], how='left')
wlp = pd.merge(left=picks, right=wl, left_on=['SitID'], right_on=['SitID'], how='left')
wlp.columns

Index([u'SitID', u'HorID', u'Pick', u'Quality', u'UWI (AGS)', u'UWI', u'lng',
       u'lat'],
      dtype='object')

In [12]:
wlp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20327 entries, 0 to 20326
Data columns (total 8 columns):
SitID        20327 non-null object
HorID        20327 non-null object
Pick         20327 non-null float64
Quality      20327 non-null int64
UWI (AGS)    20327 non-null object
UWI          20327 non-null object
lng          20327 non-null object
lat          20327 non-null object
dtypes: float64(1), int64(1), object(6)
memory usage: 1.4+ MB


In [13]:
wlp.head()

,SitID,HorID,Pick,Quality,UWI (AGS),UWI,lng,lat
0,102496,1000,321.0,1,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
1,102496,5000,438.0,2,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
2,102496,7000,458.0,3,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
3,102496,9000,468.0,3,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907
4,102496,9500,468.0,3,0674010812000,00/12-08-067-01W4/0,-110.12932,54.785907


In [14]:
# las files
path_logs = home + "/Logs"
las = [x for x in os.listdir(path_logs + "/") if x.endswith(".LAS")]
print("# LAS files: " + str(len(las)))

# LAS files: 2171


In [15]:
# a sample of curves from a log file
wellname = 'AB-07-12-093-10W4-0'
lasio.read(path_logs + "/" + wellname + ".LAS").df().head()

,ILD,GR,CALI,NPHI
DEPT,,,,
3.00,4.785,40.188,318.972,0.532
3.25,4.277,42.542,320.911,0.563
3.50,6.303,43.705,320.850,0.547
3.75,3.963,40.679,321.774,0.467
4.00,3.413,31.046,322.714,0.481


### Reading LAS Files with lasio

In [16]:
las = [lasio.read(path_logs + "/" + filename) for filename in os.listdir(path_logs) if filename.endswith(".LAS")]
type(las), type(las[0])

No handlers could be found for logger "lasio.las"


(list, lasio.las.LASFile)

In [17]:
las[2153].header['Well']

[HeaderItem(mnemonic=WELL, unit=, value=AMOCO AOSTRA GLISPLH1, descr=Well_nam...),
 HeaderItem(mnemonic=LOC, unit=, value=AB/05-02-086-07W4/0, descr=Location   ...),
 HeaderItem(mnemonic=UWI, unit=, value=AB/05-02-086-07W400, descr=Uwi        ...),
 HeaderItem(mnemonic=ENTR, unit=, value=KATHY, descr=Entered      - ENTERED BY),
 HeaderItem(mnemonic=SRVC, unit=, value=SCHLUMBERGER, descr=Scn          - SE...),
 HeaderItem(mnemonic=STRT, unit=M, value=160, descr=top_depth    - START DEPTH),
 HeaderItem(mnemonic=STOP, unit=M, value=235.001, descr=bot_depth    - STOP D...),
 HeaderItem(mnemonic=STEP, unit=M, value=0.3, descr=increment    - STEP LENGTH),
 HeaderItem(mnemonic=NULL, unit=, value=-999.25, descr=NULL Value)]

In [18]:
las[2153].header['Parameter']

[HeaderItem(mnemonic=GL, unit=M, value=478.6, descr=gl           - GROUND LEV...),
 HeaderItem(mnemonic=EREF, unit=M, value=480, descr=kb           - ELEVATION ...),
 HeaderItem(mnemonic=DATM, unit=M, value=480, descr=datum        - DATUM ELEV...),
 HeaderItem(mnemonic=TDD, unit=M, value=241, descr=tdd          - TOTAL DEPTH...),
 HeaderItem(mnemonic=RUN, unit=, value=ONE, descr=Run          - RUN NUMBER),
 HeaderItem(mnemonic=ENG, unit=, value=BILL, descr=Engineer     - RECORDING E...),
 HeaderItem(mnemonic=WIT, unit=, value=JIM, descr=Witness      - WITNESSED BY),
 HeaderItem(mnemonic=MUD, unit=, value=GEL, descr=Mud_type     - MUD TYPE),
 HeaderItem(mnemonic=MATR:1, unit=, value=LIMESTONE, descr=Logunit      - NEU...),
 HeaderItem(mnemonic=TMAX, unit=C, value=60, descr=BHT          - MAXIMUM REC...),
 HeaderItem(mnemonic=BHTD, unit=M, value=241, descr=BHTDEP       - MAXIMUM RE...),
 HeaderItem(mnemonic=MATR:2, unit=, value=1, descr=LOGUNIT      - NEUTRON MATRIX),
 HeaderItem(mnemoni

In [19]:
las[2153].header['Version']

[HeaderItem(mnemonic=VERS, unit=, value=2.0, descr=CWLS LOG ASCII STANDARD -V...),
 HeaderItem(mnemonic=WRAP, unit=, value=NO, descr=ONE LINE PER DEPTH STEP)]

In [20]:
las[2153].header['Curves']

[CurveItem(mnemonic=DEPT, unit=M, value=00 001 00 00, descr=DEPTH        - DEPTH, original_mnemonic=DEPT, data.shape=(251L,)),
 CurveItem(mnemonic=GR:1, unit=API, value=00 310 00 00, descr=GR           - GAMMA RAY, original_mnemonic=GR, data.shape=(251L,)),
 CurveItem(mnemonic=DPHI:1, unit=V/V, value=00 890 00 00, descr=PHID         - DENSITY POROSITY (LIMESTONE), original_mnemonic=DPHI, data.shape=(251L,)),
 CurveItem(mnemonic=ILD:1, unit=OHMM, value=00 120 00 00, descr=RESD         - DEEP RESISTIVITY (DIL), original_mnemonic=ILD, data.shape=(251L,)),
 CurveItem(mnemonic=GR:2, unit=API, value=00 310 00 00, descr=GR           - GAMMA RAY, original_mnemonic=GR, data.shape=(251L,)),
 CurveItem(mnemonic=DPHI:2, unit=V/V, value=00 890 00 00, descr=PHID         - DENSITY POROSITY (LIMESTONE), original_mnemonic=DPHI, data.shape=(251L,)),
 CurveItem(mnemonic=ILD:2, unit=OHMM, value=00 120 00 00, descr=RESD         - DEEP RESISTIVITY (DIL), original_mnemonic=ILD, data.shape=(251L,))]

In [21]:
len(las[2153].header['Curves']['DEPT'].data)

251

In [22]:
las[2153].header['Curves']['DEPT'].data[0:10]

array([ 160. ,  160.3,  160.6,  160.9,  161.2,  161.5,  161.8,  162.1,
        162.4,  162.7])

In [23]:
# type of curves (logs) present in the las file
las[2153].header['Curves'].keys()

[u'DEPT', u'GR:1', u'DPHI:1', u'ILD:1', u'GR:2', u'DPHI:2', u'ILD:2']

In [24]:
# function to retrieve UWI (location of the UWI may vary within the LAS file)
keywds = ['UWI', 'UWI:1', 'UWI:2', 'UWI:3']
def getKeyValue(idx, keywds):
    for keywd in keywds:
        if (keywd) not in las[idx].header["Well"]:
            pass
        else:
            return [keywd, las[idx].header["Well"][keywd].value]

In [25]:
# counting more common curves
curves = []
for i in range(len(las)):
    df_las = las[i].df()
    curves.extend(list(df_las.columns))

In [26]:
# ordered set of curves across all logs files
counter = collections.Counter(curves)
counter = collections.OrderedDict(sorted(counter.items()))
print(counter.keys())
print(counter.values())

[u'CALI', u'COND', u'DELT', u'DENS', u'DPHI', u'DPHI:1', u'DPHI:2', u'DT', u'GR', u'GR:1', u'GR:2', u'IL', u'ILD', u'ILD:1', u'ILD:2', u'ILM', u'LITH', u'LLD', u'LLS', u'NPHI', u'PHID', u'PHIN', u'RESD', u'RHOB', u'RT', u'SFL', u'SFLU', u'SN', u'SNP', u'SP']
[783, 3, 98, 4, 1917, 1, 1, 14, 2169, 1, 1, 2, 2154, 1, 1, 6, 1, 2, 1, 2008, 8, 4, 6, 132, 1, 3, 6, 1, 2, 14]


In [27]:
las[2153].header['Well']['UWI'].value

u'AB/05-02-086-07W400'

In [28]:
las[2153].header['Curves'].keys()

[u'DEPT', u'GR:1', u'DPHI:1', u'ILD:1', u'GR:2', u'DPHI:2', u'ILD:2']

In [29]:
las[2153].header['Curves']

[CurveItem(mnemonic=DEPT, unit=M, value=00 001 00 00, descr=DEPTH        - DEPTH, original_mnemonic=DEPT, data.shape=(251L,)),
 CurveItem(mnemonic=GR:1, unit=API, value=00 310 00 00, descr=GR           - GAMMA RAY, original_mnemonic=GR, data.shape=(251L,)),
 CurveItem(mnemonic=DPHI:1, unit=V/V, value=00 890 00 00, descr=PHID         - DENSITY POROSITY (LIMESTONE), original_mnemonic=DPHI, data.shape=(251L,)),
 CurveItem(mnemonic=ILD:1, unit=OHMM, value=00 120 00 00, descr=RESD         - DEEP RESISTIVITY (DIL), original_mnemonic=ILD, data.shape=(251L,)),
 CurveItem(mnemonic=GR:2, unit=API, value=00 310 00 00, descr=GR           - GAMMA RAY, original_mnemonic=GR, data.shape=(251L,)),
 CurveItem(mnemonic=DPHI:2, unit=V/V, value=00 890 00 00, descr=PHID         - DENSITY POROSITY (LIMESTONE), original_mnemonic=DPHI, data.shape=(251L,)),
 CurveItem(mnemonic=ILD:2, unit=OHMM, value=00 120 00 00, descr=RESD         - DEEP RESISTIVITY (DIL), original_mnemonic=ILD, data.shape=(251L,))]

In [30]:
# Selecting only those wells which have common curves: DEPT; DPHI; GR; ILD; NPHI
keys = ['DEPT', 'DPHI', 'DPHI:1', 'DPHI:2', 'GR', 'GR:1', 'GR:2', 'ILD', 'ILD:1', 'ILD:2', \
        'NPHI', 'NPHI:1', 'NPHI:2', 'PHIN', 'PHIN:1', 'PHIN:2']
values = ['DEPT', 'DPHI', 'DPHI', 'DPHI', 'GR', 'GR', 'GR', 'ILD', 'ILD', 'ILD', \
          'NPHI', 'NPHI', 'NPHI', 'NPHI', 'NPHI', 'NPHI']
dictOfCurves = dict(zip(keys, values))

columns = ['i', 'key_uwi', 'uwi', 'curvesNew', 'curvesOld']
rows = []

In [31]:
# find ':' on curve header name (Note: maybe not required)
import re
def findColon(name):
    m = re.search("\:", name)
    if (m is not None):
        return m.span()[0]

In [32]:
# create dataframe with standardized set of curves [GR, DPHI, ILD, DEPT, NPHI]
for i in range(len(las)):
    las_curves = las[i].header['Curves'].keys()
    c = [str(dictOfCurves[c]) for c in keys if c in las_curves]
    key_uwi, uwi = getKeyValue(i, keywds)
    rows.append([i, key_uwi, uwi, list(set(c)), las_curves])
    
df = pd.DataFrame(rows, columns=columns)
rows = []
df = df.reset_index(drop=True)

In [33]:
df.head()

,i,key_uwi,uwi,curvesNew,curvesOld
0,0,UWI,00/01-01-073-05W5/0,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, DPHI, NPHI, GR, CALI, ILD]"
1,1,UWI,00/01-01-095-19W4/0,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, GR, CALI, NPHI, DPHI, ILD]"
2,2,UWI,00/01-03-085-15W400,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, DPHI, NPHI, GR, ILD]"
3,3,UWI,00/01-04-075-23W4/0,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, ILD, NPHI, DPHI, GR, CALI]"
4,4,UWI,00/01-05-085-15W400,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, GR, NPHI, DPHI, ILD]"


In [34]:
len(df)

2171

In [35]:
# drop rows with less than 5 curves including DEPT
dfl = df[df['curvesNew'].map(len) > 4]
dfl = dfl.reset_index(drop=True)

In [36]:
len(dfl)

1848

In [37]:
dfl.head()

,i,key_uwi,uwi,curvesNew,curvesOld
0,0,UWI,00/01-01-073-05W5/0,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, DPHI, NPHI, GR, CALI, ILD]"
1,1,UWI,00/01-01-095-19W4/0,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, GR, CALI, NPHI, DPHI, ILD]"
2,2,UWI,00/01-03-085-15W400,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, DPHI, NPHI, GR, ILD]"
3,3,UWI,00/01-04-075-23W4/0,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, ILD, NPHI, DPHI, GR, CALI]"
4,4,UWI,00/01-05-085-15W400,"[DEPT, ILD, NPHI, GR, DPHI]","[DEPT, GR, NPHI, DPHI, ILD]"


In [38]:
# final dataframe with wells having these curves [DEPT, DPHI, GR, ILD, NPHI]
dl = pd.DataFrame([], columns=['UWI', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI'])
dl.columns

for i in range(len(dfl)):
    well_num = dfl.iloc[i]['i']
    uwi_type = dfl.iloc[i]['key_uwi']
    uwi = las[well_num].header['Well'][uwi_type].value

    nrecords = len(las[well_num].header['Curves']['DEPT'].data)

    UWI = pd.Series([uwi] * nrecords)
    DEPT = pd.Series(las[well_num].header['Curves']['DEPT'].data)
    DPHI = pd.Series(las[well_num].header['Curves']['DPHI'].data)
    GR = pd.Series(las[well_num].header['Curves']['GR'].data)
    ILD = pd.Series(las[well_num].header['Curves']['ILD'].data)
    NPHI = pd.Series(las[well_num].header['Curves']['NPHI'].data)

    d = {
        'UWI': UWI,
        'DEPT': DEPT,
        'DPHI': DPHI,
        'GR': GR,
        'ILD': ILD,
        'NPHI': NPHI
    }
    dl = dl.append(pd.DataFrame(d), ignore_index=True)

In [39]:
dl.head()

,DEPT,DPHI,GR,ILD,NPHI,UWI
0,390.00,0.199,82.478,2.923,0.457,00/01-01-073-05W5/0
1,390.25,0.208,86.413,2.925,0.456,00/01-01-073-05W5/0
2,390.50,0.246,90.229,2.917,0.452,00/01-01-073-05W5/0
3,390.75,0.266,90.944,2.898,0.475,00/01-01-073-05W5/0
4,391.00,0.287,88.866,2.890,0.484,00/01-01-073-05W5/0


In [40]:
# get UWI (AGS) using a not-well-formed  UWI
def getUWIAGS(wrongUWI):

    s1 = wrongUWI[9:12]
    s2 = wrongUWI[16:17]
    s3 = wrongUWI[13:15]
    s4 = wrongUWI[6:8]
    s5 = wrongUWI[3:5]
    s6 = wrongUWI[0:2]
    s7 = wrongUWI[18:19]
    
    return(s1 + s2 + s3 + s4 + s5 + s6 + s7)

In [41]:
getUWIAGS('AD/05-15-088-08W400')

'0884081505AD0'

In [42]:
len(dl)

1474751

In [43]:
(dl.iloc[1474750].UWI, getUWIAGS(dl.iloc[1474750].UWI))

(u'AD/05-15-088-08W400', u'0884081505AD0')

In [44]:
# adding UWI_AGS column to join table Wellst which has the SitID. SitID is common to all data tables
dl['UWI_AGS'] = [getUWIAGS(dl.iloc[i].UWI) for i in range(len(dl))]

In [45]:
dl.head()

,DEPT,DPHI,GR,ILD,NPHI,UWI,UWI_AGS
0,390.00,0.199,82.478,2.923,0.457,00/01-01-073-05W5/0,0735050101000
1,390.25,0.208,86.413,2.925,0.456,00/01-01-073-05W5/0,0735050101000
2,390.50,0.246,90.229,2.917,0.452,00/01-01-073-05W5/0,0735050101000
3,390.75,0.266,90.944,2.898,0.475,00/01-01-073-05W5/0,0735050101000
4,391.00,0.287,88.866,2.890,0.484,00/01-01-073-05W5/0,0735050101000


In [46]:
# see how this 'not-well-formed UWI': AD/05-15-088-08W400, should be this: AD/05-15-088-08W4/0
dl.tail()

,DEPT,DPHI,GR,ILD,NPHI,UWI,UWI_AGS
1474746,142.00,0.039,64.370,50.572,0.149,AD/05-15-088-08W400,0884081505AD0
1474747,142.25,0.034,64.406,51.559,0.152,AD/05-15-088-08W400,0884081505AD0
1474748,142.50,0.032,60.770,56.706,0.156,AD/05-15-088-08W400,0884081505AD0
1474749,142.75,0.039,53.479,61.181,0.148,AD/05-15-088-08W400,0884081505AD0
1474750,143.00,0.030,48.631,61.220,0.150,AD/05-15-088-08W400,0884081505AD0


In [47]:
(wellst.columns, wellst.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2193 entries, 0 to 2192
Data columns (total 3 columns):
SitID        2193 non-null object
UWI (AGS)    2193 non-null object
UWI          2193 non-null object
dtypes: object(3)
memory usage: 51.5+ KB


(Index([u'SitID', u'UWI (AGS)', u'UWI'], dtype='object'), None)

In [48]:
# merging dataframe 'dl' with logs and wells table (wellst) to get SitID  
dl = pd.merge(dl, wellst[['SitID', 'UWI (AGS)']], left_on='UWI_AGS', right_on='UWI (AGS)', how='left')
dt = dl[['SitID', 'DEPT', 'DPHI', 'GR', 'ILD', 'NPHI']]
dt.head()

,SitID,DEPT,DPHI,GR,ILD,NPHI
0,142218,390.00,0.199,82.478,2.923,0.457
1,142218,390.25,0.208,86.413,2.925,0.456
2,142218,390.50,0.246,90.229,2.917,0.452
3,142218,390.75,0.266,90.944,2.898,0.475
4,142218,391.00,0.287,88.866,2.890,0.484


In [49]:
dt.tail()

,SitID,DEPT,DPHI,GR,ILD,NPHI
1474746,117119,142.00,0.039,64.370,50.572,0.149
1474747,117119,142.25,0.034,64.406,51.559,0.152
1474748,117119,142.50,0.032,60.770,56.706,0.156
1474749,117119,142.75,0.039,53.479,61.181,0.148
1474750,117119,143.00,0.030,48.631,61.220,0.150


In [70]:
# merging dataframe 'dt' with 'wlp' on SitID and [DEPT/Pick]
m = pd.merge(left=dt, right=wlp, left_on=['SitID', 'DEPT'], right_on=['SitID', 'Pick'], how='left')
# change HorID to string
m['HorID'] = m['HorID'].astype(str)
# drop NA values
m = m.dropna()
m.head()

,SitID,DEPT,DPHI,GR,ILD,NPHI,HorID,Pick,Quality,UWI (AGS),UWI,lng,lat
12,142218,393.0,0.299,72.249,3.338,0.429,1000,393.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
636,142218,549.0,0.247,76.644,4.313,0.427,4000,549.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
776,142218,584.0,0.184,72.472,5.379,0.357,5000,584.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
828,142218,597.0,0.180,69.942,4.265,0.367,7000,597.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691
868,142218,607.0,0.191,57.457,9.279,0.312,11000,607.0,1.0,0735050101000,00/01-01-073-05W5/0,-114.620656,55.288691


In [69]:
m = m.reset_index(drop=True)
m['Id'] = m.index.values
m = m[['Id', 'SitID', 'DPHI', 'GR', 'ILD', 'NPHI', 'lng', 'lat', 'HorID', 'Pick']]
m.to_csv('m_out.csv', sep="\t", index=None)

### end of Create tidy dataset